In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import os.path as op
from scipy.io import loadmat

In [2]:
data_dir = '/home/rob/data/MEG'
fname_fwd_model = 'gainConstrainedSub05.mat'
fname_inv_model = 'inverseConstrainedSub05.mat'
fname_noise_cov = 'noiseCovarianceSub05.mat'

In [3]:
fwd_model = loadmat(op.join(data_dir, fname_fwd_model))
G = fwd_model['Gain_constrained'][:306, :]
print('Shape of G =', np.shape(G))

noise = loadmat(op.join(data_dir, fname_noise_cov))
C = noise['NM'][0, 0][0][:306, :306]
print('Shape of C =', np.shape(C))

R = np.identity(np.size(G, 1))
print('Shape of R =', np.shape(R))

lambda2 = 1 / 3 ** 2

Shape of G = (306, 15002)
Shape of C = (306, 306)
Shape of R = (15002, 15002)


# Load the BST inverse solution

In [ ]:
inv_model = loadmat(op.join(data_dir, fname_inv_model))
M0 = inv_model['ImagingKernel']
print('Shape of M_BST =', np.shape(M0))

AAinv0 = G @ M0

plt.figure()
plt.imshow(AAinv0)
plt.colorbar()
plt.show()

# Straightforward method

In [ ]:
M = R @ G.T @ np.linalg.inv(G @ R @ G.T + lambda2 * C)

In [ ]:
AAinv = G @ M

plt.figure()
plt.imshow(AAinv)
plt.colorbar()
plt.show()

# Computationally efficient method

In [4]:
Lc, Uc = np.linalg.eig(C)
whitener = np.diag(list(1 / lc for lc in Lc)) @ Uc.T

G_tilde = whitener @ G
print('Shape of G_tilde =', np.shape(G_tilde))

Shape of G_tilde = (306, 15002)


In [ ]:
countNonDiag = np.count_nonzero(R - np.diag(np.diagonal(R)))

if countNonDiag == 0:
    R_sqrt = np.diag(list(np.sqrt(r) for r in np.diagonal(R)))
else:
    R_sqrt = np.linalg.cholesky(R)
    
print('Shape of R_sqrt =', np.shape(R_sqrt))

In [5]:
A = G_tilde @ R_sqrt

U, L, VT = np.linalg.svd(A, full_matrices=0)

print('Shape of U =', np.shape(U))
print('Shape of L =', np.shape(L))
print('Shape of V =', np.shape(VT.T))

Shape of R_sqrt = (15002, 15002)


In [7]:
Gamma = np.diag(list(lk ** 2 / (lk * (lk ** 2 + lambda2)) for lk in L))
print('Shape of Gamma =', np.shape(Gamma))

Shape of Gamma = (306, 306)


In [8]:
M_tilde = R_sqrt @ VT.T @ Gamma @ U.T
print('Shape of M_tilde =', np.shape(M_tilde))

Shape of M_tilde = (15002, 306)


In [10]:
M_eff = M_tilde @ whitener
print('Shape of M_eff =', np.shape(M_eff))

Shape of M_eff = (15002, 306)


In [13]:
AAinv_eff = G @ M_eff
# AAinv_tilde = G_tilde @ M_tilde

In [14]:
plt.figure()
plt.imshow(AAinv_eff.real)
plt.colorbar()
plt.show()